In [ ]:
import os
import pandas as pd
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_colwidth', -1)
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')

In [ ]:
# Import relevant dataframes:

main_table = pd.read_csv(r"C:\Users\Public\_Data\_Data\Latest_Version\Main_Data\Main_Table_10182019.csv")
print(len(main_table))
faculty_table = pd.read_csv(r"C:\Users\Public\_Data\_Data\Latest_Version\Faculty_Data\Faculty_Table_08272019.csv")
print(len(faculty_table))
skill_table = pd.read_csv(r"C:\Users\Public\_Data\_Data\Latest_Version\Skills_Data\Skill_Table_06072019.csv")
print(len(skill_table))

In [ ]:
def title_swap(string):
    dictionary = {'Community and Social Services':'Counseling and Religious Life',
                  'Customer and Client Support':'Online Support and University Information',
                  'Hospitality, Food, and Tourism':'Event Management and Hospitality', 
                  'Planning and Analysis':'Analysis', 
                  'Curriculum and Instructional Designer / Developer':'Curriculum and Instructional Designer', 
                  'Special Education Teacher':'Accessibility and Disability Services', 
                  'Teaching Assistant':'Faculty Support', 
                  'Tutor':'Academic Tutor',
                  'Clerical and Administrative':'Administrative',
                  'na' : 'Uncategorized'}
    if(string in dictionary):
        return dictionary[string]
    else:
        return string
    
def val_year(year1, year2):
    if(year1 >= year2):
        if(year1>year2):
            temp = year1
            year1 = year2
            year2 = temp
        else:
            raise ValueError('Years cannot be the same.')
            
    return (year1, year2)

def cat_validate(actual, desired_list):
    return actual in desired_list

In [ ]:
post_doc = main_table[main_table['Job Title'].str.contains('postdoc', case=False, na=False) |
                      main_table['Job Title'].str.contains('post doc', case=False, na=False)|
                      main_table['Job Title'].str.contains('post-doc', case=False, na=False)]
print(len(post_doc))

In [ ]:
mask = faculty_table[faculty_table['Post-Doctoral'] == 1][['Job ID']]
print(len(mask))

In [ ]:
boolean_vec = (main_table['Job Title'].str.contains('postdoc', case=False, na=False) |
                      main_table['Job Title'].str.contains('post doc', case=False, na=False)|
                      main_table['Job Title'].str.contains('post-doc', case=False, na=False))

main_table['post_doc'] = boolean_vec + 0

faculty_table.merge(main_table[['Job ID', 'post_doc']], on='Job ID', how='inner')